In [2]:
%pip install contractions pandas

import pandas as pd

# Load datasets
df = pd.read_csv("combined_dataset.csv", encoding='utf-8')  # First dataset


# Print sample data
print(df.head())
print(df['hate'].value_counts())  # Check label distribution




[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
                                                text  hate
0   Are bhajans played daily.??  5 times a day..?? 🤔     1
1  @zainabsikander Perfectly said.... 100% agreed...     1
2  Tats sad no mother thinks her baby as puppy bu...     1
3  Speechs wont work if ill thoughts still around...     1
4  @timesofindia Mamta is doing wrong. But where ...     0
hate
0    9507
1    9497
Name: count, dtype: int64


In [3]:
%pip install contractions nltk numpy scikit-learn



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import re
import pandas as pd
import contractions

def clean_text(text):
    # Expand contractions
    text = contractions.fix(text)
    # Remove @mentions and hashtags
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'#\w+', '', text)
    # Remove URLs
    text = re.sub(r'http\S+|www.\S+', '', text)
    # Remove non-ASCII characters (e.g., emojis)
    text = text.encode('ascii', 'ignore').decode('ascii')
    # Remove extra whitespace and non-alphabetic characters
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    # Convert to lowercase
    text = text.lower()
    return text

# Load your dataset (ensure the CSV has a 'text' column)
df = pd.read_csv("combined_dataset.csv")

# Apply cleaning function to each text entry
df['cleaned_text'] = df['text'].apply(clean_text)

# Save the cleaned texts into a corpus file, one sentence per line
with open("hinglish_corpus.txt", "w", encoding="utf8") as f:
    for text in df['cleaned_text']:
        if text.strip():  # Write non-empty lines only
            f.write(text.strip() + "\n")


In [5]:
%pip install fasttext
import fasttext

model = fasttext.train_unsupervised('hinglish_corpus.txt', model='skipgram')
model.save_model("hinglish_embeddings.bin")
model = fasttext.load_model("hinglish_embeddings.bin")



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Read 0M words
Number of words:  6445
Number of labels: 0
Progress: 100.0% words/sec/thread:  123569 lr:  0.000000 avg.loss:  2.463794 ETA:   0h 0m 0s


In [6]:

import pandas as pd
import contractions
import string
import re
import time
import nltk
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [7]:
# Download stopwords
nltk.download('wordnet')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

#data preprocessing
def clean_text(text):
    text = contractions.fix(text)
    text = re.sub(r'@\w+', '', text)
    # text = re.sub(r'#\w+', '', text)
    text = re.sub(r'http\S+|www.\S+', '', text)
    text = text.encode('ascii', 'ignore').decode('ascii')
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    text = text.lower()
    tokens = text.split()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    return " ".join(tokens)


df['cleaned_text'] = df['text'].apply(clean_text)

# Feature extraction: BoW(uni to tri)(binary)
vectorizer = CountVectorizer(ngram_range=(1,4),binary=True)
X = vectorizer.fit_transform(df['cleaned_text'])
y = df['hate']

# Feature selection using Mutual information classification
selector = SelectKBest(score_func=chi2, k=40000)
X_selected = selector.fit_transform(X, y)
# Stratified train-test split
from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_idx, test_idx in sss.split(X_selected, y):
    X_train, X_test = X_selected[train_idx], X_selected[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]


# Model training with GridSearch for smoothing (epsilon / alpha)
parameters = {'alpha': list(np.arange(0.1, 4, 0.05))}
nb = MultinomialNB()
clf = GridSearchCV(nb, parameters, cv=5)
clf.fit(X_train, y_train)
best_model = clf.best_estimator_

# Evaluation
start = time.time()
y_pred = best_model.predict(X_test)
end = time.time()

print("Best alpha (ε):", clf.best_params_['alpha'])
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred, target_names=['Not Hate', 'Hate']))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print(f"Testing time per statement: {(end - start) / len(y_test) * 1e6:.4f} µs")

# 10-fold cross-validation
cv_scores = cross_val_score(best_model, X_selected, y, cv=10)
print("10-fold CV Accuracy: %.4f" % cv_scores.mean())
print("95%% CI: (%.4f, %.4f)" % (cv_scores.mean() - 2 * cv_scores.std(), cv_scores.mean() + 2 * cv_scores.std()))


[nltk_data] Downloading package wordnet to /home/kxngh/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/kxngh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Best alpha (ε): 0.1
Accuracy: 0.8650355169692187
Classification Report:
               precision    recall  f1-score   support

    Not Hate       0.85      0.89      0.87      1902
        Hate       0.88      0.84      0.86      1899

    accuracy                           0.87      3801
   macro avg       0.87      0.87      0.86      3801
weighted avg       0.87      0.87      0.86      3801

Confusion Matrix:
 [[1692  210]
 [ 303 1596]]
Testing time per statement: 0.1673 µs
10-fold CV Accuracy: 0.8718
95% CI: (0.8565, 0.8871)


In [8]:
import re, time, contractions
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import StratifiedShuffleSplit, GridSearchCV, StratifiedKFold
from sklearn.pipeline import Pipeline, FunctionTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# ── 1. Download NLTK resources ───────────────────────────────────────
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
STOP_WORDS = set(stopwords.words('english'))
lemmatizer  = WordNetLemmatizer()

# ── 2. Cleaning function ─────────────────────────────────────────────
def clean_text_series(texts):
    cleaned = []
    for doc in texts:
        txt = contractions.fix(str(doc))                             # expand contractions
        txt = re.sub(r'@\w+|http\S+|www.\S+', ' ', txt)               # strip mentions/URLs
        txt = re.sub(r"[^a-zA-Z\s]", ' ', txt).lower()               # keep letters + lowercase
        toks = [lemmatizer.lemmatize(tok) for tok in txt.split()
                if tok not in STOP_WORDS]                           # lemmatize & drop stopwords
        cleaned.append(' '.join(toks))
    return np.array(cleaned)

cleaner = FunctionTransformer(clean_text_series, validate=False)

# ── 3. Prepare data & split ─────────────────────────────────────────
df['cleaned_text'] = clean_text_series(df['text'])
X_raw = df['cleaned_text']
y     = df['hate'].values

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.20, random_state=42)
train_idx, test_idx = next(sss.split(X_raw, y))
X_train, X_test = X_raw[train_idx], X_raw[test_idx]
y_train, y_test = y[train_idx], y[test_idx]

# ── 4. Build pipeline ────────────────────────────────────────────────
pipeline = Pipeline([
    ('clean', cleaner),
    ('vect', CountVectorizer(ngram_range=(1,4), binary=True)),
    ('select', SelectKBest(score_func=mutual_info_classif, k=40000)),
    ('clf',   MultinomialNB())
])

param_grid = {
    'clf__alpha': [0.001, 0.01, 0.1, 0.5, 1, 2, 5, 10]
}

# ── 5. Nested CV & hyperparameter search ────────────────────────────
cv_inner = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
search   = GridSearchCV(pipeline, param_grid,
                        scoring='f1_macro',
                        cv=cv_inner,
                        n_jobs=-1,
                        verbose=1)
search.fit(X_train, y_train)

print("Best α:", search.best_params_['clf__alpha'])

# ── 6. Final evaluation ──────────────────────────────────────────────
best_model = search.best_estimator_
y_pred     = best_model.predict(X_test)

print("\nTest-set performance:")
print(f"Accuracy:  {accuracy_score(y_test, y_pred):.4f}")
print(classification_report(y_test, y_pred,
      target_names=['Not Hate','Hate'], digits=4))               # :contentReference[oaicite:9]{index=9}
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


[nltk_data] Downloading package stopwords to /home/kxngh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/kxngh/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Fitting 5 folds for each of 8 candidates, totalling 40 fits
Best α: 2

Test-set performance:
Accuracy:  0.7077
              precision    recall  f1-score   support

    Not Hate     0.7132    0.6956    0.7043      1902
        Hate     0.7025    0.7199    0.7111      1899

    accuracy                         0.7077      3801
   macro avg     0.7078    0.7077    0.7077      3801
weighted avg     0.7078    0.7077    0.7077      3801

Confusion Matrix:
 [[1323  579]
 [ 532 1367]]


In [1]:
import re
import time
import numpy as np
import pandas as pd
import contractions
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import VotingClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import StandardScaler
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import nltk
from imblearn.over_sampling import SMOTE
import joblib
import warnings
warnings.filterwarnings('ignore')

# Download necessary NLTK data
nltk.download('wordnet', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('punkt', quiet=True)
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# Enhanced preprocessing function with sentiment features
def advanced_clean_text(text):
    if not isinstance(text, str):
        return ""
    
    # Fix contractions
    text = contractions.fix(text)
    
    # Remove mentions
    text = re.sub(r'@\w+', '', text)
    
    # Keep hashtag content
    text = re.sub(r'#(\w+)', r'\1', text)
    
    # Remove URLs
    text = re.sub(r'http\S+|www.\S+', '', text)
    
    # Convert to ASCII
    text = text.encode('ascii', 'ignore').decode('ascii')
    
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    
    # Keep only letters and spaces, convert to lowercase
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    text = text.lower()
    
    # Tokenize using NLTK for better handling
    tokens = word_tokenize(text)
    
    # Lemmatize and remove stopwords
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]
    
    return " ".join(tokens)

# Additional feature extraction: text length, uppercase ratio, etc.
def extract_text_features(text):
    if not isinstance(text, str) or len(text) == 0:
        return [0, 0, 0, 0]
    
    # Original text before cleaning for some features
    original_text = text
    
    # Number of words
    num_words = len(original_text.split())
    
    # Ratio of uppercase letters
    if len(original_text) > 0:
        uppercase_ratio = sum(1 for c in original_text if c.isupper()) / len(original_text)
    else:
        uppercase_ratio = 0
    
    # Number of exclamation marks
    exclamation_count = original_text.count('!')
    
    # Number of question marks
    question_count = original_text.count('?')
    
    return [num_words, uppercase_ratio, exclamation_count, question_count]

print("Preprocessing text data...")
df['cleaned_text'] = df['text'].apply(advanced_clean_text)

# Extract additional text features
print("Extracting text features...")
text_features = np.array([extract_text_features(text) for text in df['text']])

# Feature extraction: Create both word count and TF-IDF features
print("Creating feature extraction pipelines...")

# Pipeline for TF-IDF features
tfidf_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(
        ngram_range=(1, 3),
        min_df=3,
        max_df=0.9,
        sublinear_tf=True,
        use_idf=True,
        max_features=40000
    )),
    ('selector', SelectKBest(score_func=chi2, k=25000))
])

# Pipeline for Count Vector features (binary)
count_pipeline = Pipeline([
    ('count', CountVectorizer(
        ngram_range=(1, 4),
        binary=True,
        min_df=2,
        max_df=0.95
    )),
    ('selector', SelectKBest(score_func=chi2, k=25000))
])

# Prepare data
X_text = df['cleaned_text']
y = df['hate']

# Create stratified train/test split
X_train_text, X_test_text, X_train_features, X_test_features, y_train, y_test = train_test_split(
    X_text, text_features, y, test_size=0.2, random_state=42, stratify=y
)

# Transform text features using both pipelines
print("Transforming features...")
X_train_tfidf = tfidf_pipeline.fit_transform(X_train_text)
X_test_tfidf = tfidf_pipeline.transform(X_test_text)

X_train_count = count_pipeline.fit_transform(X_train_text)
X_test_count = count_pipeline.transform(X_test_text)

# Concatenate features from both vectorizers with additional text features
from scipy.sparse import hstack
X_train_combined = hstack([X_train_tfidf, X_train_count, X_train_features])
X_test_combined = hstack([X_test_tfidf, X_test_count, X_test_features])

# Check for class imbalance
class_distribution = np.bincount(y_train) / len(y_train)
print(f"Class distribution in training set: {class_distribution}")

# Apply SMOTE for mild oversampling of minority class if needed
if abs(class_distribution[0] - class_distribution[1]) > 0.05:
    print("Applying SMOTE to balance classes...")
    smote = SMOTE(random_state=42, sampling_strategy=0.95)  # not completely balanced
    X_train_combined, y_train = smote.fit_resample(X_train_combined, y_train)
    print(f"New class distribution: {np.bincount(y_train) / len(y_train)}")

# Define individual models
print("Training models...")
nb = MultinomialNB(alpha=0.05)  # Using your best alpha
lr = LogisticRegression(C=5, max_iter=1000, class_weight='balanced', random_state=42)
svm = LinearSVC(C=1, class_weight='balanced', dual=False, max_iter=10000, random_state=42)
rf = RandomForestClassifier(n_estimators=100, max_depth=50, min_samples_leaf=2, class_weight='balanced', random_state=42)

# Train individual models for comparison
start_time = time.time()
nb.fit(X_train_combined, y_train)
y_pred_nb = nb.predict(X_test_combined)
print(f"\nNaiveBayes - Accuracy: {accuracy_score(y_test, y_pred_nb):.4f}, F1: {f1_score(y_test, y_pred_nb):.4f}")

lr.fit(X_train_combined, y_train)
y_pred_lr = lr.predict(X_test_combined)
print(f"LogisticRegression - Accuracy: {accuracy_score(y_test, y_pred_lr):.4f}, F1: {f1_score(y_test, y_pred_lr):.4f}")

svm.fit(X_train_combined, y_train)
y_pred_svm = svm.predict(X_test_combined)
print(f"SVM - Accuracy: {accuracy_score(y_test, y_pred_svm):.4f}, F1: {f1_score(y_test, y_pred_svm):.4f}")

rf.fit(X_train_combined, y_train)
y_pred_rf = rf.predict(X_test_combined)
print(f"RandomForest - Accuracy: {accuracy_score(y_test, y_pred_rf):.4f}, F1: {f1_score(y_test, y_pred_rf):.4f}")

# Create voting ensemble with soft voting
print("\nTraining ensemble model...")
ensemble = VotingClassifier(
    estimators=[
        ('nb', nb),
        ('lr', lr),
        ('svm', svm),
        ('rf', rf)
    ],
    voting='soft',  # Use probabilities for better results
    weights=[2, 1, 1, 1]  # Give more weight to NB since it performed well initially
)

# For soft voting with LinearSVC (which doesn't implement predict_proba)
from sklearn.calibration import CalibratedClassifierCV
svm_calibrated = CalibratedClassifierCV(svm, cv=5)
svm_calibrated.fit(X_train_combined, y_train)

# Replace SVM with calibrated version for soft voting
ensemble.estimators_[2] = ('svm', svm_calibrated)

# Train ensemble
ensemble.fit(X_train_combined, y_train)
end_time = time.time()
print(f"Training time: {end_time - start_time:.2f} seconds")

# Evaluate ensemble
y_pred_ensemble = ensemble.predict(X_test_combined)
accuracy = accuracy_score(y_test, y_pred_ensemble)
precision = precision_score(y_test, y_pred_ensemble)
recall = recall_score(y_test, y_pred_ensemble)
f1 = f1_score(y_test, y_pred_ensemble)

print(f"\nEnsemble Model Results:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred_ensemble, target_names=['Not Hate', 'Hate']))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_ensemble))

# 10-fold cross-validation on the best individual model (use the fastest one for CV)
print("\nPerforming cross-validation on NaiveBayes model...")
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
cv_scores = cross_val_score(nb, X_train_combined, y_train, cv=cv)
print(f"10-fold CV Accuracy: {cv_scores.mean():.4f}")
print(f"95% CI: ({cv_scores.mean() - 2 * cv_scores.std():.4f}, {cv_scores.mean() + 2 * cv_scores.std():.4f})")

# Save the best model (either ensemble or individual based on performance)
best_model = ensemble if accuracy_score(y_test, y_pred_ensemble) > accuracy_score(y_test, y_pred_nb) else nb
print(f"\nSaving {'ensemble' if best_model == ensemble else 'NaiveBayes'} as best model...")

# Save necessary components for prediction
model_data = {
    'tfidf_pipeline': tfidf_pipeline,
    'count_pipeline': count_pipeline,
    'model': best_model,
}
joblib.dump(model_data, 'advanced_hate_speech_model.pkl')
print("Model saved as 'advanced_hate_speech_model.pkl'")

# Prediction function for new texts
def predict_hate_speech(text, model_data=model_data):
    # Clean the text
    cleaned_text = advanced_clean_text(text)
    
    # Extract additional features
    text_features = extract_text_features(text)
    
    # Transform using both pipelines
    tfidf_features = model_data['tfidf_pipeline'].transform([cleaned_text])
    count_features = model_data['count_pipeline'].transform([cleaned_text])
    
    # Combine all features
    combined_features = hstack([tfidf_features, count_features, [text_features]])
    
    # Make prediction
    prediction = model_data['model'].predict(combined_features)[0]
    
    # Get probability if possible
    try:
        probability = model_data['model'].predict_proba(combined_features)[0][1]
    except:
        probability = None
    
    return {
        'is_hate_speech': bool(prediction),
        'probability': probability,
        'cleaned_text': cleaned_text
    }

# Test the prediction function
print("\nExample classification:")
examples = [
    "I love how diverse and inclusive our community has become!",
    "Those people are all the same, they should go back to where they came from",
    "The weather is nice today, I might go for a walk"
]

for example in examples:
    result = predict_hate_speech(example)
    print(f"\nText: {example}")
    print(f"Prediction: {'Hate Speech' if result['is_hate_speech'] else 'Not Hate Speech'}")
    if result['probability'] is not None:
        print(f"Probability: {result['probability']:.4f}")

Preprocessing text data...


NameError: name 'df' is not defined